# STIS `specutils` Loader

[github](https://github.com/astropy/specutils)  
[specutils documentation](https://specutils.readthedocs.io/)  

2018 Mar 28  

In [1]:
import os
import glob
import six
import numpy as np
import warnings

from astropy.io import fits
from astropy.table import Table, Column
from astropy.units import Unit as u
from astropy.wcs import WCS
from astropy.nddata import StdDevUncertainty

%matplotlib inline
from matplotlib import pyplot as plt

from specutils.io.registers import data_loader
import specutils
from specutils import Spectrum1D

In [2]:
# Define an optional identifier. If made specific enough, this circumvents the
# need to add `format="my-format"` in the `Spectrum1D.read` call.
def identify_stis_fits(origin, *args, **kwargs):
    if isinstance(args[0], six.string_types) and (os.path.splitext(args[0].lower())[1] == '.fits'):
        with fits.open(args[0]) as f:
            return f[0].header.get('INSTRUME', '') == 'STIS'
    return False

In [3]:
# Don't let AstroPy yell at us about the poor formatting of STIS data:
warnings.filterwarnings('ignore', append=True, 
                        message='.*contains multiple slashes, which is discouraged by the FITS standard.*')
warnings.filterwarnings('ignore', append=True, 
                        message=".*Changed units: 'angstrom' -> 'Angstrom'.*")


@data_loader('stis-format', identifier=identify_stis_fits)
def stis_fits(file_name, ext=1, sdqflags=None, weight=None, **kwargs):
    '''
    file_name
        Path + name of input STIS FITS file
    ext
        
    '''
    name = os.path.basename(file_name.rstrip(os.sep)).rsplit('.', 1)[0]

    with fits.open(file_name, **kwargs) as f:
        header = f[0].header
        
        if not sdqflags:
            sdqflags = f[ext].header.get('SDQFLAGS', 31743)
        
        tab = Table.read(file_name, hdu=ext)
        # Not entirely necessary:
        #tab.remove_columns([x for x in tab.colnames if x not in ['FLUX', 'WAVELENGTH', 'ERROR', 'DQ', 'SPORDER']])
        
        # Determine the FLT file matching the input X1D/SX1 file (look in same directory):
        dirname, basename = os.path.split(file_name)
        search_for = [
            os.path.join(dirname, basename.rsplit('_x1c.fits',1)[0] + '_flt.fits'),
            os.path.join(dirname, basename.rsplit('_s1c.fits',1)[0] + '_flt.fits'),
            os.path.join(dirname, basename.rsplit('_x1d.fits',1)[0] + '_flt.fits'),
            os.path.join(dirname, basename.rsplit('_sx1.fits',1)[0] + '_flt.fits')]
        search_for.sort(key=len)
        search_for = search_for[0]
        flt_file = glob.glob(search_for)
        
        if len(flt_file) != 1:
            print ('WARNING:  Unique FLT file not found in X1D\'s directory:  {}'.format(search_for))
            wcs = None
        else:
            flt_file = flt_file[0]
            # WCS must be generated from the FLT file:
            wcs = WCS(fits.getheader(flt_file, ext=('SCI',ext)))
        
        if weight is not None:
            # Require weight array to be 2D:
            weight = np.array(weight).squeeze()
            if len(np.shape(weight)) == 1:
                weight = np.array([weight])
            #print (np.shape(weight), np.shape(tab['WAVELENGTH']))
            if np.shape(tab['WAVELENGTH']) != np.shape(weight):
                raise Exception('Dimensionality of (optional) WEIGHT array must match '
                                'WAVELENGTH/FLUX/ERROR/DQ arrays.')
        
        spectra = []  # Will become a spectrumCollection later!
        for i in range(len(tab)):
            flags = tab[i]['DQ'] & sdqflags
            uncertainty = StdDevUncertainty(tab[i]['ERROR']) #* u(header[1].data.columns['ERROR'].unit)
            wave = tab[i]['WAVELENGTH'] * u(f[1].data.columns['WAVELENGTH'].unit.replace('Angstroms', 'Angstrom'))
            flux = tab[i]['FLUX'] * u(f[1].data.columns['FLUX'].unit)
            meta = {'header': header, 'sdqflags': sdqflags, 'flags': flags, 'dq': tab[i]['DQ'], 
                    'sporder': tab[i]['SPORDER']}
            if weight is not None:
                meta['weight'] = weight[i,:]
            
            # WCS shouldn't overtake spectral_axis here!  Leave out WCS for now:
            spectra.append(Spectrum1D(spectral_axis=wave, flux=flux, wcs=None, uncertainty=uncertainty, meta=meta))

    return spectra

In [5]:
weight = np.zeros((57, 1024), dtype=np.float)
for i in range(np.shape(weight)[0]):
    weight[i,:] += i
s1 = stis_fits('/Users/lockwood/Desktop/splice_data/obik03020_x1d.fits', weight=weight)

s2 = stis_fits('/Users/lockwood/Desktop/combining_spectra/data/ocr7nc080_x1d.fits')

s3 = stis_fits('/Users/lockwood/Desktop/combining_spectra/data/ocr7nc080_x1d.fits', weight=np.ones(1024))

#s4 = Spectrum1D.read('/Users/lockwood/Desktop/combining_spectra/data/ocr7nc080_x1d.fits', format='stis-format')
#s = Spectrum1D.read('/Users/lockwood/Desktop/combining_spectra/data/ocr7nc080_x1d.fits')

#s1[0].wavelength, s1[1].wavelength
s1

[Spectrum1D([ -1.24896556e-12,  -7.49990998e-13,  -6.78731214e-13, ...,
               1.55404530e-12,   1.16925794e-12,  -2.12088094e-12]),
 Spectrum1D([ -1.23642242e-12,  -7.62700233e-13,  -7.71511273e-13, ...,
               1.05388886e-12,   8.33566885e-13,  -1.35168602e-12]),
 Spectrum1D([ -1.13982391e-12,  -7.27311007e-13,  -6.07973687e-13, ...,
               6.94110622e-13,   3.54556608e-13,  -3.60146982e-12]),
 Spectrum1D([ -1.03531940e-12,  -7.94681920e-13,  -5.58888818e-13, ...,
               1.45131270e-12,   1.50131838e-12,  -1.22557455e-12]),
 Spectrum1D([  1.61853712e-12,   2.18995113e-12,   2.81485902e-12, ...,
               1.40227586e-12,   1.32120519e-12,  -1.50001658e-12]),
 Spectrum1D([  1.46370589e-12,   2.37586730e-12,   2.40482872e-12, ...,
               1.13375563e-13,  -4.07383301e-13,  -2.88258003e-12]),
 Spectrum1D([  1.77068245e-12,   1.81024835e-12,   2.33507137e-12, ...,
               1.65325696e-12,   1.01678464e-12,   4.16102184e-13]),
 Spectrum1D([

In [1]:
from astropy.modeling.models import Gaussian1D

In [2]:
g = Gaussian1D()

In [3]:
g

<Gaussian1D(amplitude=1., mean=0., stddev=1.)>

In [4]:
print(g)

Model: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Parameters:
    amplitude mean stddev
    --------- ---- ------
          1.0  0.0    1.0
